In [ ]:
import dask
import xarray as xr
import numpy as np
import pandas as pd
import dask.array as da
import ocetrac
import warnings
warnings.filterwarnings('ignore')
from datetime import date
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

In [2]:
dir_path = "/pub/hfdrake/datasets/CM4_MHW_blobs/"
mt_path = "/pub/mariant3/WarmWaterMasses/02_tracking-mhws"
ds = xr.open_mfdataset(f"{dir_path}/data/ocean_daily_cmip*.nc", chunks={'time':100})
region_tos = ds['tos'].sel(xh=slice(-130, -70), yh=slice(8, 38))
#region_tos = ds['tos'].sel(xh=slice(-100, -73), yh=slice(17, 35)) #GOM + Caribbean
#region_tos = ds['tos'].sel(xh=slice(-100, -81), yh=slice(18, 31)) #GOM

## Calculate climatology for the region

In [5]:
doy = region_tos['time'].dt.dayofyear
region_climatology = region_tos.chunk(dict(time=-1)).groupby(doy).mean(dim='time')
region_climatology

<xarray.DataArray 'tos' (dayofyear: 365, yh: 132, xh: 236)>
dask.array<stack, shape=(365, 132, 236), dtype=float32, chunksize=(1, 132, 236), chunktype=numpy.ndarray>
Coordinates:
  * xh         (xh) float64 -129.8 -129.6 -129.3 -129.1 ... -70.67 -70.41 -70.16
  * yh         (yh) float64 8.222 8.469 8.716 8.963 ... 37.3 37.5 37.69 37.89
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 ... 359 360 361 362 363 364 365
Attributes:
    long_name:      Sea Surface Temperature
    units:          degC
    cell_methods:   area:mean yh:mean xh:mean time: mean
    cell_measures:  area: areacello
    time_avg_info:  average_T1,average_T2,average_DT
    standard_name:  sea_surface_temperature

In [6]:
#Reassigning dates
climatology_dates = xr.DataArray(
    region_climatology.sel(dayofyear=region_tos["time"].sel(time=slice("0186","0189")).dt.dayofyear), #.sel for the dates
    dims=["time", "yh", "xh"],
    coords={"time": region_tos["time"].sel(time=slice("0186","0189")), "xh": region_tos["xh"], "yh": region_tos["yh"]}
)
#Uncomment below to load
#climatology_dates = climatology_dates.load()

<xarray.DataArray 'tos' (time: 1460, yh: 132, xh: 236)>
dask.array<getitem, shape=(1460, 132, 236), dtype=float32, chunksize=(1, 132, 236), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 0186-01-01 12:00:00 ... 0189-12-31 12:00:00
  * xh       (xh) float64 -129.8 -129.6 -129.3 -129.1 ... -70.67 -70.41 -70.16
  * yh       (yh) float64 8.222 8.469 8.716 8.963 9.21 ... 37.3 37.5 37.69 37.89
Attributes:
    long_name:      Sea Surface Temperature
    units:          degC
    cell_methods:   area:mean yh:mean xh:mean time: mean
    cell_measures:  area: areacello
    time_avg_info:  average_T1,average_T2,average_DT
    standard_name:  sea_surface_temperature

In [280]:
#Define and start and end dates to run through ocetrac
start = climatology_dates.get_index("time").get_loc("0186-03-15").start #"0186-04-15"
end = climatology_dates.get_index("time").get_loc("0186-11-02").start #"0186-11-02"

In [282]:
#ocetrac procedure up to Tracker
hot_water = climatology_dates.isel(time=slice(start,end)) > 29
mask_ocean = 1 * np.ones(region_tos.shape[1:]) * np.isfinite(region_tos.isel(time=0))
mask_land = 0 * np.ones(region_tos.shape[1:]) * np.isnan(region_tos.isel(time=0))
mask = mask_ocean + mask_land
Tracker = ocetrac.Tracker(hot_water, mask, radius=3, min_size_quartile=0.75, timedim='time', xdim='xh', ydim='yh', positive=True)

In [262]:
#Run ocetrac
blobs = Tracker.track()

minimum area: 1058.25
inital objects identified 	 60
final objects tracked 	 1


In [ ]:
#climatology_dates.to_netcdf("/pub/mariant3/WarmWaterMasses/data/climatology/climatology-manso-tos-0186-01-01-0189-12-31.nc")

#### 95th percentile

In [22]:
per95 = region_tos.chunk(dict(time=-1)).groupby(doy).quantile(0.95, dim='time')
per95

<xarray.DataArray 'tos' (dayofyear: 365, yh: 132, xh: 236)>
dask.array<concatenate, shape=(365, 132, 236), dtype=float64, chunksize=(1, 132, 236), chunktype=numpy.ndarray>
Coordinates:
  * xh         (xh) float64 -129.8 -129.6 -129.3 -129.1 ... -70.67 -70.41 -70.16
  * yh         (yh) float64 8.222 8.469 8.716 8.963 ... 37.3 37.5 37.69 37.89
    quantile   float64 0.95
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 ... 359 360 361 362 363 364 365

In [24]:
per95_dates = xr.DataArray(
    per95.sel(dayofyear=region_tos['time'].sel(time=slice("0186","0189")).dt.dayofyear),
    dims=["time", "yh", "xh"],
    coords={"time": region_tos['time'].sel(time=slice("0186","0189")), "xh": region_tos["xh"], "yh": region_tos["yh"]}
)
per95_dates

<xarray.DataArray 'tos' (time: 1460, yh: 132, xh: 236)>
dask.array<getitem, shape=(1460, 132, 236), dtype=float64, chunksize=(1, 132, 236), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 0186-01-01 12:00:00 ... 0189-12-31 12:00:00
  * xh       (xh) float64 -129.8 -129.6 -129.3 -129.1 ... -70.67 -70.41 -70.16
  * yh       (yh) float64 8.222 8.469 8.716 8.963 9.21 ... 37.3 37.5 37.69 37.89

In [26]:
#per95_dates.to_netcdf("/pub/mariant3/WarmWaterMasses/data/per95-manso-0186-01-01-0189-12-31.nc")